In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.12:0.14.0 pyspark-shell'

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Streamming").getOrCreate()

:: loading settings :: url = jar:file:/usr/lib/spark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.databricks#spark-xml_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a04c3821-da06-4122-9a7b-c7c5b8bdc615;1.0
	confs: [default]
	found com.databricks#spark-xml_2.12;0.14.0 in central
	found commons-io#commons-io;2.8.0 in central
	found org.glassfish.jaxb#txw2;2.3.4 in central
	found org.apache.ws.xmlschema#xmlschema-core;2.2.5 in central
:: resolution report :: resolve 288ms :: artifacts dl 7ms
	:: modules in use:
	com.databricks#spark-xml_2.12;0.14.0 from central in [default]
	commons-io#commons-io;2.8.0 from central in [default]
	org.apache.ws.xmlschema#xmlschema-core;2.2.5 from central in [default]
	org.glassfish.jaxb#txw2;2.3.4 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|

## Stream Emitter

In [3]:
from pyspark.streaming import StreamingContext

ssc = StreamingContext(spark.sparkContext, batchDuration=5)
inputRDD = ssc.textFileStream("hdfs://hw2-m/user/root/p1t3_test")
def saveAsTextFile(inputRDD):
    if not inputRDD.isEmpty():
        df = inputRDD.map(lambda x:x.split('\t')).toDF(['article', 'rank'])
        df.coalesce(1).write.option("delimiter", "\t").mode("append").csv("hdfs://hw2-m/user/root/p2t2_streaming")
    else:
        print('Waiting...')
inputRDD.foreachRDD(saveAsTextFile)

## Stream Receiver

In [4]:
from pyspark.sql.types import StructType

userSchema = StructType().add("article", "string").add("rank", "double")
df = spark.readStream \
    .option("sep", "\t") \
    .schema(userSchema) \
    .csv("hdfs://hw2-m/user/root/p2t2_streaming")

In [5]:
df = df.select('article', 'rank').where("rank > 0.5")

In [6]:
# Read the data output to a new directory
rawdata = spark.read.option("delimiter", "\t").csv('hdfs://hw2-m/user/root/p1t3_whole')

In [7]:
# Start the emitter
ssc.start()

22/04/29 18:34:50 WARN org.apache.spark.streaming.StreamingContext: Dynamic Allocation is enabled for this application. Enabling Dynamic allocation for Spark Streaming applications can cause data loss if Write Ahead Log is not enabled for non-replayable sources. See the programming guide for details on how to enable the Write Ahead Log.


Waiting...


In [8]:
# Emit the data to a new directory
rawdata.repartition(10).write.option("delimiter", "\t").csv('p1t3_test')

22/04/29 18:35:00 ERROR org.apache.spark.streaming.scheduler.JobScheduler: Error generating jobs for time 1651257300000 ms
org.apache.hadoop.mapreduce.lib.input.InvalidInputException: Input path does not exist: hdfs://hw2-m/user/root/p1t3_test/_temporary
	at org.apache.hadoop.mapred.LocatedFileStatusFetcher.getFileStatuses(LocatedFileStatusFetcher.java:153)
	at org.apache.hadoop.mapreduce.lib.input.FileInputFormat.listStatus(FileInputFormat.java:280)
	at org.apache.hadoop.mapreduce.lib.input.FileInputFormat.getSplits(FileInputFormat.java:396)
	at org.apache.spark.rdd.NewHadoopRDD.getPartitions(NewHadoopRDD.scala:131)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:300)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:296)
	at org.apache.spark.streaming.dstream.FileInputDStream.$anonfun$filesToRDD$1(FileInputDStream.scala:289)
	at scala.collection.TraversableLike.$anonfun$map$1(TraversableLike.scala:286)
	at scala.collection.Ind

Waiting...
Waiting...


Waiting...


22/04/29 18:35:20 WARN org.apache.hadoop.util.concurrent.ExecutorHelper: Thread (Thread[GetFileInfo #1,5,main]) interrupted: 
java.lang.InterruptedException
	at com.google.common.util.concurrent.AbstractFuture.get(AbstractFuture.java:510)
	at com.google.common.util.concurrent.FluentFuture$TrustedFuture.get(FluentFuture.java:88)
	at org.apache.hadoop.util.concurrent.ExecutorHelper.logThrowableFromAfterExecute(ExecutorHelper.java:48)
	at org.apache.hadoop.util.concurrent.HadoopThreadPoolExecutor.afterExecute(HadoopThreadPoolExecutor.java:90)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1157)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:750)
22/04/29 18:35:20 WARN org.apache.hadoop.util.concurrent.ExecutorHelper: Thread (Thread[GetFileInfo #1,5,main]) interrupted: 
java.lang.InterruptedException
	at com.google.common.util.concurrent.AbstractFuture.get(AbstractFuture.java:510)
	at c

In [9]:
df.writeStream \
    .format("csv") \
    .option("delimiter", "\t") \
    .option("checkpointLocation", "hdfs://hw2-m/user/root/p2t2_output") \
    .option("path", "hdfs://hw2-m/user/root/p2t2_output") \
    .start()

22/04/29 18:35:20 WARN org.apache.spark.sql.streaming.StreamingQueryManager: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


22/04/29 18:35:25 ERROR org.apache.spark.streaming.scheduler.JobScheduler: Error generating jobs for time 1651257325000 ms
org.apache.hadoop.mapreduce.lib.input.InvalidInputException: Input path does not exist: hdfs://hw2-m/user/root/p1t3_test/_SUCCESS
	at org.apache.hadoop.mapred.LocatedFileStatusFetcher.getFileStatuses(LocatedFileStatusFetcher.java:153)
	at org.apache.hadoop.mapreduce.lib.input.FileInputFormat.listStatus(FileInputFormat.java:280)
	at org.apache.hadoop.mapreduce.lib.input.FileInputFormat.getSplits(FileInputFormat.java:396)
	at org.apache.spark.rdd.NewHadoopRDD.getPartitions(NewHadoopRDD.scala:131)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:300)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:296)
	at org.apache.spark.streaming.dstream.FileInputDStream.$anonfun$filesToRDD$1(FileInputDStream.scala:289)
	at scala.collection.TraversableLike.$anonfun$map$1(TraversableLike.scala:286)
	at scala.collection.Index

Waiting...
Waiting...
Waiting...
Waiting...
Waiting...


Waiting...


Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...


In [10]:
ssc.stop()